# Make EGFR Immune Figure (7A)

This notebooks takes all the trans genes that have the same results and runs a GSEA using NCI-Nature_2016. Top hits included T and B cell signaling pathways. Heat map includes genes from GSEA as well as additional genes realted to the immune system. 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas.util.testing as tm
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.utils as u
import plot_utils as p 
import warnings
warnings.filterwarnings('ignore')

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Step 1: Find Trans proteins with opposite effects in different cancers 

Load df with all of the genes that are FDR significant.

In [3]:
FDR_sig = pd.read_csv("csv_files/Supplemental_Table_EGFR_sig_only.csv")
FDR_sig = FDR_sig.set_index("Comparison")
FDR_sig

,Correlation_GBM,P_value_GBM,Correlation_ccRCC,P_value_ccRCC,Correlation_LUAD,P_value_LUAD,Correlation_LSCC,P_value_LSCC,Correlation_BR,P_value_BR,Correlation_HNSCC,P_value_HNSCC
Comparison,,,,,,,,,,,,
PHLDA1,0.789551,9.649849e-19,NaN,NaN,NaN,NaN,0.682116,1.398013e-12,NaN,NaN,0.583064,1.395609e-08
CDH4,0.656331,4.744022e-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GRB2,-0.589008,3.310325e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.481605,2.053150e-05
PHLDA3,0.561528,2.006861e-06,0.364164,0.005159,NaN,NaN,0.651618,5.103796e-11,NaN,NaN,0.693125,2.351012e-14
GLA,-0.562315,2.006861e-06,-0.273040,0.047412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
IGLV2-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.277801,4.834886e-02
SCN10A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.364870,4.918329e-02
PCYT1A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.245333,4.976210e-02


In [4]:
def HasPosNeg(row):
    hasPos = False
    hasNeg= False

    for item in row:
        if pd.isnull(item):
            continue
        if item < 0:
            hasNeg = True
        if item > 0:
            hasPos = True
            
    if hasPos & hasNeg:
        return True
    return False

def CountPosNeg(row):
    hasPos = False
    hasNeg= False
    counter = 0
    for item in row:
        if pd.isnull(item):
            continue
        if item < -0:
            hasNeg = True
            counter += 1
        if item > 0:
            hasPos = True
            counter += 1
    return counter


Subset data frame to include only trans genes that have the same effects in different cancers by using apply function

In [5]:
col = ["Correlation_GBM","Correlation_ccRCC","Correlation_BR","Correlation_LUAD","Correlation_HNSCC","Correlation_LSCC"]
FDR_corr = FDR_sig[col]
FDR_corr["Pos_Neg"] = FDR_corr.apply(HasPosNeg, axis = 1)

FDR_corr_False = FDR_corr[FDR_corr['Pos_Neg']==False]


In [6]:
#Get only proteins significant in atleast two cancers
FDR_corr_False["Num_corr"] = FDR_corr_False.apply(CountPosNeg, axis = 1)
FDR_corr_False.sort_values(by=['Num_corr'],ascending=False)
FDR_corr_False = FDR_corr_False[FDR_corr_False['Num_corr'].isin([2,3,4,5,6])]
FDR_corr_False.head()

,Correlation_GBM,Correlation_ccRCC,Correlation_BR,Correlation_LUAD,Correlation_HNSCC,Correlation_LSCC,Pos_Neg,Num_corr
Comparison,,,,,,,,
PHLDA1,0.789551,NaN,NaN,NaN,0.583064,0.682116,False,3
GRB2,-0.589008,NaN,NaN,NaN,-0.481605,NaN,False,2
PHLDA3,0.561528,0.364164,NaN,NaN,0.693125,0.651618,False,4
GLA,-0.562315,-0.273040,NaN,NaN,NaN,NaN,False,2
SEC23IP,-0.543668,NaN,-0.417054,-0.284303,NaN,NaN,False,3


The manuscript mentions that there are 1136 proteins with the same directional effect. Here is the derivation of that number. 

In [7]:
same_sign_prot = FDR_corr_False.index.tolist()
print("Total number of trans proteins with same effects in different cancers is " + str(len(same_sign_prot)))

Total number of trans proteins with same effects in different cancers is 1139


The manuscript mentions pathways enriched in these 1136 proteins. Here is the enrichment analysis results

# Run GSEA

In [8]:
same_sign_enr = gp.enrichr(gene_list = same_sign_prot, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='test/enrichr_NCI-Nature')
same_sign_enr.res2d.head(10)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,NCI-Nature_2016,TCR signaling in naive CD4+ T cells Homo sapie...,17/64,6.689900e-08,0.000010,0,0,6.065119,100.196267,MAP4K1;CHUK;PRKCB;WAS;VAV1;ZAP70;CD4;LCK;GRAP2...
1,NCI-Nature_2016,HIF-1-alpha transcription factor network Homo ...,17/66,1.095129e-07,0.000010,0,0,5.816945,93.229475,NCOA1;EGLN1;JUN;SMAD4;PFKFB3;TFRC;SERPINE1;SLC...
2,NCI-Nature_2016,Regulation of RAC1 activity Homo sapiens 351aa...,12/38,7.471561e-07,0.000036,0,0,7.713467,108.813810,ARHGAP9;EPS8;BCR;ABR;TRIO;RAP1GDS1;ELMO1;ARHGA...
3,NCI-Nature_2016,CXCR4-mediated signaling events Homo sapiens 4...,20/100,7.472291e-07,0.000036,0,0,4.195934,59.191594,YES1;RALB;ITGA4;MAPKAP1;ITGA2;PXN;LIMK1;GNAI3;...
4,NCI-Nature_2016,Validated transcriptional targets of AP1 famil...,11/35,2.288981e-06,0.000087,0,0,7.653923,99.404587,FOSL1;GJA1;JUN;ITGB4;PLAU;MMP1;MGP;NFATC2;JUNB...
5,NCI-Nature_2016,TCR signaling in naive CD8+ T cells Homo sapie...,13/53,6.162173e-06,0.000196,0,0,5.432349,65.172332,CHUK;PRKCB;VAV1;ZAP70;LCK;GRAP2;MAP3K8;LCP2;CS...
6,NCI-Nature_2016,Integrin family cell surface interactions Homo...,9/26,7.895481e-06,0.000206,0,0,8.828527,103.728304,ITGA4;ITGB4;ITGA2;ITGB8;ITGAV;ITGA6;ITGB6;ITGA...
7,NCI-Nature_2016,Direct p53 effectors Homo sapiens 67c3b75d-619...,22/136,8.628360e-06,0.000206,0,0,3.238892,37.766955,FOXA1;STEAP3;JUN;PPP1R13B;GPX1;S100A2;CSE1L;HG...
8,NCI-Nature_2016,Arf6 trafficking events Homo sapiens 7a5b8f09-...,12/49,1.422770e-05,0.000302,0,0,5.417108,60.456660,ACAP1;ITGA4;ITGA2;CTNND1;CTNNA1;CTNNB1;ITGAV;I...
9,NCI-Nature_2016,Arf6 signaling events Homo sapiens 3b3d91b8-61...,10/35,1.711124e-05,0.000308,0,0,6.673516,73.247015,CYTH3;EFNA1;GNA15;ACAP1;IQSEC1;HGF;PXN;KIF13B;...


We selected proteins related to B and T cell signaling based off of enrichment results. 

In [17]:
t_only = ['CD3E', 'CD3G', 'CD4','GRAP2', 'FYB1', 'PRKCQ'] 
b_only = ['PPP3CC', 'CD79A', 'SYK', 'BTK', 'LYN', 'BLNK', 'HLA-DMA', 'HLA-DMB']
both = ['CD2', 'CD5', 'GRB2', 'LCK', 'ZAP70', 'VAV1', 'PTPN6', 'PTPRC', 'PIK3CD', 'PIK3CG', 'INPP5D',
        'PLCG1', 'PLCG2', 'NFATC2', 'PRKCB','NFKB2', 'DOCK2', 'RAC2', 'ELMO1', 'WAS']
immune = t_only + b_only + both
len(immune)

34

# Make data frame for figure 7A

In [18]:
#Get append version of the df with all cancer type, fdr sig trans results
df_FDR_append = pd.read_csv("csv_files/sig_prot_heatmap_EGFR.csv")

#subset dataframe to include genes only desired for figure 
df_FDR_append= df_FDR_append[df_FDR_append.Comparison.isin(immune)]


 Add new column that will be the unique index and order the new index. This way genes will be grouped by proteins associated with T cell, B cell, or both.

In [19]:
# Order index for Heatmap
df_FDR_append["Index"] = df_FDR_append["Comparison"] + " " + df_FDR_append["Cancer"]
df_FDR_append = df_FDR_append.set_index("Index")


ordered_list = []
cancer_list = ['BR', 'GBM', 'HNSCC', 'ccRCC', 'LSCC', 'LUAD']
for gene in immune:
    for cancer in cancer_list:
        ordered_list.append(gene+' '+cancer)
        
ordered_df = df_FDR_append.reindex(ordered_list)
ordered_df = ordered_df.loc[ordered_df['P_Value'] < 0.05] # Keep sig 


# Make Figure 7A

In [20]:
legend_min = df_FDR_append["P_Value"].min()
#Make plot using plot utils
p.plotCircleHeatMap(ordered_df, circle_var = "P_Value",color_var = "Correlation", x_axis = "Comparison", y_axis = "Cancer", plot_width= 1000, plot_height = 500, legend_min = legend_min, legend_max = 0.05, font_size = 10, show_legend = True , save_png = "png_files/Figure7A.png")

Loading BokehJS ...

Figure(id='1760', ...)

# Check if blanks are due to no data 

The follow code chunks show that the following cancers/genes don't have data: colon THBD, Kidney MMP12, and Ovarian MMP12.

In [12]:
#Get append version of the df with all proteins 
df_all_prot_append = pd.read_csv("Make_Tables/csv_files/all_prot_heatmap_EGFR.csv")

#subset dataframe to include genes only desired for figure 
df_all_immune = df_all_prot_append[df_all_prot_append.Comparison.isin(immune)]
# With 8 cancer types and 34 genes there should be 272 rows. 24 genes are missing
print("Number of rows in data frame " + str(len(df_all_immune)))


Number of rows in data frame 248


In [13]:
def find_missing_genes(test_list, full_list):
    missing_list = []
    for gene in full_list:
        if (gene not in test_list):
            missing_list.append(gene)
    return missing_list

print list of missing genes

In [14]:
cancers = ["GBM", "LUAD","LSCC","HNSCC","OV","ccRCC","BR","CO"]
fig_genes =set(ordered_df.Comparison.to_list())
for cancer in cancers:
    cancer_check = df_all_immune[df_all_immune["Cancer"] == cancer]
    cancer_check = cancer_check.Comparison.to_list()
    print(cancer)
    print(find_missing_genes(cancer_check,immune))

GBM
['CD3E', 'CD3G', 'CD79A', 'CD5', 'LCK']
LUAD
[]
LSCC
[]
HNSCC
[]
OV
['CD3G', 'CD79A', 'CD2', 'LCK']
ccRCC
['CD79A', 'BTK', 'NFKB2']
BR
['FYB1', 'LYN']
CO
['CD3E', 'CD3G', 'GRAP2', 'FYB1', 'PRKCQ', 'PPP3CC', 'CD79A', 'BLNK', 'PIK3CD', 'PIK3CG']


In [23]:
enrich_df = same_sign_enr.res2d
p53_genes = enrich_df.iloc[7,9]

p53_genes = p53_genes.split(';')
p53_genes

['FOXA1',
 'STEAP3',
 'JUN',
 'PPP1R13B',
 'GPX1',
 'S100A2',
 'CSE1L',
 'HGF',
 'IGFBP3',
 'CAV1',
 'SERPINE1',
 'SH2D1A',
 'HTT',
 'NDRG1',
 'BBC3',
 'PYCARD',
 'CARM1',
 'TP53BP2',
 'BCL2',
 'SFN',
 'MET',
 'TP63']

In [24]:
#Get append version of the df with all cancer type, fdr sig trans results
df_FDR_append = pd.read_csv("csv_files/sig_prot_heatmap_EGFR.csv")

In [25]:
df_FDR_append= df_FDR_append[df_FDR_append.Comparison.isin(p53_genes)]
df_FDR_append

,Comparison,Correlation,P_Value,Cancer
47,HGF,-0.489587,2.025406e-04,GBM
411,PYCARD,-0.359990,6.539696e-03,GBM
546,GPX1,-0.341734,1.057417e-02,GBM
661,CSE1L,0.330816,1.346322e-02,GBM
1138,PPP1R13B,0.288964,3.590087e-02,GBM
1381,SERPINE1,0.492118,1.133625e-05,HNSCC
1384,NDRG1,0.485138,1.800834e-05,HNSCC
1514,CSE1L,0.405231,4.501477e-04,HNSCC
1684,JUN,0.364643,1.901418e-03,HNSCC
1828,S100A2,0.343751,3.581396e-03,HNSCC


In [26]:
legend_min = df_FDR_append["P_Value"].min()
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_append, circle_var = "P_Value",color_var = "Correlation", x_axis = "Comparison", y_axis = "Cancer", plot_width= 1000, plot_height = 500, legend_min = legend_min, legend_max = 0.05, font_size = 10, show_legend = True)

Loading BokehJS ...

Figure(id='1169', ...)